In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReadAllFilesInFolder").getOrCreate()
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
# Thêm các thư viện Spark cần thiết

spark = SparkSession.builder.getOrCreate()

In [2]:
data_neg = spark.read.text("D:/BinhLuan/data_train/train/neg").limit(5000)
data_pos = spark.read.text("D:/BinhLuan/data_train/train/pos").limit(5000)

data_neg_bage = data_neg.withColumn("label", lit("neg"))
data_pos_bage = data_pos.withColumn("label", lit("pos"))

data = data_neg_bage.union(data_pos_bage)

data.show()

+--------------------+-----+
|               value|label|
+--------------------+-----+
|ESCAPE_GAME : CÁI...|  neg|
|Ít khi mình phàn_...|  neg|
|Vừa đi ăn buffet ...|  neg|
|Thứ nhất , về mặt...|  neg|
|Thứ_hai , về mặt ...|  neg|
|Thứ_ba , về chất_...|  neg|
|Tương_ớt , tương ...|  neg|
|Lúc mình về , thấ...|  neg|
|Hôm_nay đi ăn bò ...|  neg|
|Chỉ có chuyện này...|  neg|
|Rồi đang tung_tẩy...|  neg|
|Rồi bạn ý chạy ra...|  neg|
|Đang ' nói_chuyện...|  neg|
|Mình cũng có nói ...|  neg|
|Rất xin_lỗi anh s...|  neg|
|Nếu em không nhầm...|  neg|
|Mình mà là con_tr...|  neg|
|Mình thật_sự muốn...|  neg|
|Số_là tôi và các ...|  neg|
|Tất_nhiên hội chú...|  neg|
+--------------------+-----+
only showing top 20 rows



In [3]:
data.describe()

DataFrame[summary: string, value: string, label: string]

In [4]:
data.describe().show()

+-------+--------------------+-----+
|summary|               value|label|
+-------+--------------------+-----+
|  count|               10000|10000|
|   mean|                null| null|
| stddev|                null| null|
|    min|! ! ! Sẽ quay lại...|  neg|
|    max|😑 điểm trừ là cơ...|  pos|
+-------+--------------------+-----+



In [5]:
from pyspark.sql.functions import trim

# Xóa khoảng trắng dư thừa trong cột "value"
data = data.withColumn("value", trim(data["value"]))

# Loại bỏ các hàng rỗng
data = data.na.drop()

# In kết quả
data.show()

+--------------------+-----+
|               value|label|
+--------------------+-----+
|ESCAPE_GAME : CÁI...|  neg|
|Ít khi mình phàn_...|  neg|
|Vừa đi ăn buffet ...|  neg|
|Thứ nhất , về mặt...|  neg|
|Thứ_hai , về mặt ...|  neg|
|Thứ_ba , về chất_...|  neg|
|Tương_ớt , tương ...|  neg|
|Lúc mình về , thấ...|  neg|
|Hôm_nay đi ăn bò ...|  neg|
|Chỉ có chuyện này...|  neg|
|Rồi đang tung_tẩy...|  neg|
|Rồi bạn ý chạy ra...|  neg|
|Đang ' nói_chuyện...|  neg|
|Mình cũng có nói ...|  neg|
|Rất xin_lỗi anh s...|  neg|
|Nếu em không nhầm...|  neg|
|Mình mà là con_tr...|  neg|
|Mình thật_sự muốn...|  neg|
|Số_là tôi và các ...|  neg|
|Tất_nhiên hội chú...|  neg|
+--------------------+-----+
only showing top 20 rows



In [6]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

#Tách câu thành các token (Tokenization) 
tokenizer = Tokenizer(inputCol="value", outputCol="words")
df = tokenizer.transform(data)

df.show()

+--------------------+-----+--------------------+
|               value|label|               words|
+--------------------+-----+--------------------+
|ESCAPE_GAME : CÁI...|  neg|[escape_game, :, ...|
|Ít khi mình phàn_...|  neg|[ít, khi, mình, p...|
|Vừa đi ăn buffet ...|  neg|[vừa, đi, ăn, buf...|
|Thứ nhất , về mặt...|  neg|[thứ, nhất, ,, về...|
|Thứ_hai , về mặt ...|  neg|[thứ_hai, ,, về, ...|
|Thứ_ba , về chất_...|  neg|[thứ_ba, ,, về, c...|
|Tương_ớt , tương ...|  neg|[tương_ớt, ,, tươ...|
|Lúc mình về , thấ...|  neg|[lúc, mình, về, ,...|
|Hôm_nay đi ăn bò ...|  neg|[hôm_nay, đi, ăn,...|
|Chỉ có chuyện này...|  neg|[chỉ, có, chuyện,...|
|Rồi đang tung_tẩy...|  neg|[rồi, đang, tung_...|
|Rồi bạn ý chạy ra...|  neg|[rồi, bạn, ý, chạ...|
|Đang ' nói_chuyện...|  neg|[đang, ', nói_chu...|
|Mình cũng có nói ...|  neg|[mình, cũng, có, ...|
|Rất xin_lỗi anh s...|  neg|[rất, xin_lỗi, an...|
|Nếu em không nhầm...|  neg|[nếu, em, không, ...|
|Mình mà là con_tr...|  neg|[mình, mà, là, co...|


In [7]:
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

#Loại bỏ Stop Word
#Đọc file stop word tiếng việt có sẵn
def read_stopwords(stopword_file):
    with open(stopword_file, 'r', encoding='utf-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords

# loại bỏ stop word dựa trên dữ liệu stop word tiếng việt ở trên
def remove_stopwords_vn(words, stopwords):
    from underthesea import word_tokenize
    tokens = word_tokenize(words)
    return [w for w in tokens if w not in stopwords]

# Load stop word file
stopword_file = 'vietnamese-stopwords.txt'
stopwords = read_stopwords(stopword_file)

# Define UDF
remove_stopwords_udf = udf(lambda x: remove_stopwords_vn(x, stopwords), ArrayType(StringType()))

#Xóa stop word
remover = StopWordsRemover(inputCol='words', outputCol='filtered_tokens', stopWords=stopwords)
df1 = remover.transform(df).drop('words')
df2 = df1.select('value', 'label', remove_stopwords_udf('value').alias('tokens'))
df2.show()

+--------------------+-----+--------------------+
|               value|label|              tokens|
+--------------------+-----+--------------------+
|ESCAPE_GAME : CÁI...|  neg|[ESCAPE_GAME, :, ...|
|Ít khi mình phàn_...|  neg|[Ít, phàn_nàn, ch...|
|Vừa đi ăn buffet ...|  neg|[Vừa, đi, buffet,...|
|Thứ nhất , về mặt...|  neg|[Thứ, ,, mặt, khô...|
|Thứ_hai , về mặt ...|  neg|[Thứ_hai, ,, mặt,...|
|Thứ_ba , về chất_...|  neg|[Thứ_ba, ,, chất_...|
|Tương_ớt , tương ...|  neg|[Tương_ớt, ,, tươ...|
|Lúc mình về , thấ...|  neg|[Lúc, ,, 1, trễ, ...|
|Hôm_nay đi ăn bò ...|  neg|[Hôm_nay, đi, bò,...|
|Chỉ có chuyện này...|  neg|[Chỉ, chia_sẻ, nó...|
|Rồi đang tung_tẩy...|  neg|[Rồi, tung_tẩy, đ...|
|Rồi bạn ý chạy ra...|  neg|[Rồi, chạy, chỗ, ...|
|Đang ' nói_chuyện...|  neg|[Đang, ', nói_chu...|
|Mình cũng có nói ...|  neg|[Mình, kia, hoàn_...|
|Rất xin_lỗi anh s...|  neg|[Rất, xin_lỗi, se...|
|Nếu em không nhầm...|  neg|[Nếu, nhầm, Tuấn,...|
|Mình mà là con_tr...|  neg|[Mình, con_trai, ...|


In [8]:
from pyspark.ml.feature import StringIndexer,VectorIndexer

#Gán nhãn cho label
label_indexer = StringIndexer(inputCol="label", outputCol="label_indexer").fit(df2)
data_transformed = label_indexer.transform(df2)
data_transformed.show()

+--------------------+-----+--------------------+-------------+
|               value|label|              tokens|label_indexer|
+--------------------+-----+--------------------+-------------+
|ESCAPE_GAME : CÁI...|  neg|[ESCAPE_GAME, :, ...|          0.0|
|Ít khi mình phàn_...|  neg|[Ít, phàn_nàn, ch...|          0.0|
|Vừa đi ăn buffet ...|  neg|[Vừa, đi, buffet,...|          0.0|
|Thứ nhất , về mặt...|  neg|[Thứ, ,, mặt, khô...|          0.0|
|Thứ_hai , về mặt ...|  neg|[Thứ_hai, ,, mặt,...|          0.0|
|Thứ_ba , về chất_...|  neg|[Thứ_ba, ,, chất_...|          0.0|
|Tương_ớt , tương ...|  neg|[Tương_ớt, ,, tươ...|          0.0|
|Lúc mình về , thấ...|  neg|[Lúc, ,, 1, trễ, ...|          0.0|
|Hôm_nay đi ăn bò ...|  neg|[Hôm_nay, đi, bò,...|          0.0|
|Chỉ có chuyện này...|  neg|[Chỉ, chia_sẻ, nó...|          0.0|
|Rồi đang tung_tẩy...|  neg|[Rồi, tung_tẩy, đ...|          0.0|
|Rồi bạn ý chạy ra...|  neg|[Rồi, chạy, chỗ, ...|          0.0|
|Đang ' nói_chuyện...|  neg|[Đang, ', nó

In [9]:
hashing_tf = HashingTF(inputCol="tokens", outputCol="HTF_features")

#tính TF 
df3 = hashing_tf.transform(data_transformed)
df3.show()

+--------------------+-----+--------------------+-------------+--------------------+
|               value|label|              tokens|label_indexer|        HTF_features|
+--------------------+-----+--------------------+-------------+--------------------+
|ESCAPE_GAME : CÁI...|  neg|[ESCAPE_GAME, :, ...|          0.0|(262144,[86002,22...|
|Ít khi mình phàn_...|  neg|[Ít, phàn_nàn, ch...|          0.0|(262144,[158,221,...|
|Vừa đi ăn buffet ...|  neg|[Vừa, đi, buffet,...|          0.0|(262144,[8195,101...|
|Thứ nhất , về mặt...|  neg|[Thứ, ,, mặt, khô...|          0.0|(262144,[9276,976...|
|Thứ_hai , về mặt ...|  neg|[Thứ_hai, ,, mặt,...|          0.0|(262144,[4686,750...|
|Thứ_ba , về chất_...|  neg|[Thứ_ba, ,, chất_...|          0.0|(262144,[402,8195...|
|Tương_ớt , tương ...|  neg|[Tương_ớt, ,, tươ...|          0.0|(262144,[3515,401...|
|Lúc mình về , thấ...|  neg|[Lúc, ,, 1, trễ, ...|          0.0|(262144,[68,10305...|
|Hôm_nay đi ăn bò ...|  neg|[Hôm_nay, đi, bò,...|          0.0|(2

In [10]:
idf = IDF(inputCol="HTF_features", outputCol="features")

#tính ITF 
idfModel = idf.fit(df3)
                   
df4 = idfModel.transform(df3)
df4.show()

+--------------------+-----+--------------------+-------------+--------------------+--------------------+
|               value|label|              tokens|label_indexer|        HTF_features|            features|
+--------------------+-----+--------------------+-------------+--------------------+--------------------+
|ESCAPE_GAME : CÁI...|  neg|[ESCAPE_GAME, :, ...|          0.0|(262144,[86002,22...|(262144,[86002,22...|
|Ít khi mình phàn_...|  neg|[Ít, phàn_nàn, ch...|          0.0|(262144,[158,221,...|(262144,[158,221,...|
|Vừa đi ăn buffet ...|  neg|[Vừa, đi, buffet,...|          0.0|(262144,[8195,101...|(262144,[8195,101...|
|Thứ nhất , về mặt...|  neg|[Thứ, ,, mặt, khô...|          0.0|(262144,[9276,976...|(262144,[9276,976...|
|Thứ_hai , về mặt ...|  neg|[Thứ_hai, ,, mặt,...|          0.0|(262144,[4686,750...|(262144,[4686,750...|
|Thứ_ba , về chất_...|  neg|[Thứ_ba, ,, chất_...|          0.0|(262144,[402,8195...|(262144,[402,8195...|
|Tương_ớt , tương ...|  neg|[Tương_ớt, ,, tươ.

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier

dtc =  DecisionTreeClassifier(featuresCol="features", labelCol="label_indexer")

# Create a pipeline to chain the preprocessing steps and the classifier together
pipeline = Pipeline(stages=[tokenizer, remover, hashing_tf, idf, dtc])

In [12]:
#chia tập dữ liệu thành train, test theo tỷ lệ 70:30
(train_data, test_data) = data_transformed.randomSplit([0.7, 0.3], seed=12345)
#chia tập dữ liệu thành train, test theo tỷ lệ 80:20
(train_data, test_data) = data_transformed.randomSplit([0.8, 0.2], seed=12345)

In [13]:
model = pipeline.fit(train_data)

In [14]:
predictions = model.transform(test_data)

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#tính toán độ chính xác
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label_indexer", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % (accuracy))

Accuracy = 0.589731


In [16]:
predictions.filter(predictions.label_indexer == 1.0).select('label', 'label_indexer', 'prediction').show()

+-----+-------------+----------+
|label|label_indexer|prediction|
+-----+-------------+----------+
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
|  pos|          1.0|       1.0|
+-----+-------------+----------+
only showing top 20 rows



In [17]:
predictions.filter(predictions.label_indexer == 0.0).select('label', 'label_indexer', 'prediction').show(1000)

+-----+-------------+----------+
|label|label_indexer|prediction|
+-----+-------------+----------+
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       0.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|          0.0|       1.0|
|  neg|   